## Objective

Here is the example how you generated a read count matrix from FACS simulation

In [32]:
using DataFrames
using Gadfly
using ColorBrewer

INFO: Precompiling module Suppressor.
This may mean module Compat does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
Stacktrace:
 [1] _require(::Symbol) at ./loading.jl:448
 [2] require(::Symbol) at ./loading.jl:398
 [3] include_from_node1(::String) at ./loading.jl:569
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /Users/hwan/.julia/v0.6/Suppressor/src/Suppressor.jl, in expression starting on line 4


LoadError: [91mFailed to precompile Suppressor to /Users/hwan/.julia/lib/v0.6/Suppressor.ji.[39m

I didn't find way to import `Crispulator` package, so I manually include several `.jl` files which I need to use. I toggled the output of below code off because many **WARNING** messages were generated, and these messages are not very informative.

In [33]:
include(joinpath(Pkg.dir("Crispulator"), "src", "run.jl"));
for file in readdir(joinpath(Pkg.dir("Crispulator"), "src/simulation/"))
    include(joinpath(Pkg.dir("Crispulator"), "src/simulation/",file));
end

INFO: Loading simulation framework


In [22]:
facs_param = FacsScreen()
facs_param.num_genes = 5000
facs_param.coverage = 10
facs_param.representation = 100
facs_param.bin_info[:bin1] = (0, 0.25)
facs_param.bin_info[:bin2] = (0.75, 1)
facs_param.seq_depth = 300
lib = Library(CRISPRn());

In [21]:
guides, guide_freqs_dist = construct_library(facs_param, lib)

cells, cell_phenotypes = transfect(facs_param, lib, guides, guide_freqs_dist)
bin_cells = select(facs_param, cells, cell_phenotypes, guides)
freqs = counts_to_freqs(bin_cells, length(guides))

seq_depths = Dict{Symbol, Int}()
for binname in keys(bin_cells)
    seq_depths[binname] = rand(Poisson(facs_param.seq_depth))
end

raw_data = sequencing(seq_depths, guides, freqs)
bc_counts, genes = differences_between_bins(raw_data);

Let's see the what is inside of `bc_count`, and you will see `counts_bin1` and `counts_bin2` are there.

In [66]:
head(bc_counts)

,gene,knockdown,theo_phenotype,obs_phenotype,behavior,class,initial_freq,counts_bin1,barcodeid,freqs_bin1,rel_freqs_bin1,freqs_bin2,counts_bin2,rel_freqs_bin2,log2fc_bin2
1,1,1.0,-0.5847940518212458,1.5315331799079293,sigmoidal,decreasing,1.3353864043282956e-5,266.5,1,1.7796327212020033e-5,1.1663019693654266,7.79436152570481e-6,117.5,0.5032119914346895,-1.2127031608457683
2,1,1.0,-0.5847940518212458,0.24368971306921755,sigmoidal,decreasing,2.9789389019631208e-5,662.5,2,4.424040066777963e-5,2.899343544857768,1.6948590381426202e-5,255.5,1.094218415417559,-1.4058255480518944
3,1,1.0,-0.5847940518212458,1.1692611210741393,sigmoidal,decreasing,1.0272203110217658e-5,244.5,3,1.6327212020033388e-5,1.0700218818380742,8.325041459369818e-6,125.5,0.537473233404711,-0.9933754856530536
4,1,1.0,-0.5847940518212458,-0.6686127470373375,sigmoidal,decreasing,1.3353864043282956e-5,335.5,4,2.2404006677796328e-5,1.4682713347921226,8.45771144278607e-6,127.5,0.5460385438972163,-1.4270439039929177
5,1,1.0,-0.5847940518212458,1.2254914506981038,sigmoidal,decreasing,5.3415456173131823e-5,1283.5,5,8.570951585976628e-5,5.617067833698031,2.6898839137645108e-5,405.5,1.7366167023554604,-1.6935378610097376
6,1,1.0,-0.5847940518212458,-0.6330632740161447,sigmoidal,decreasing,7.601430301561067e-5,1704.5,6,0.00011382303839732888,7.4595185995623625,4.076285240464345e-5,614.5,2.63169164882227,-1.5030920684188764


In [65]:
sgRNA_readcount = bc_counts[[:gene,:barcodeid,:counts_bin1,:counts_bin2]];
# a pseudocount of 0.5 added to every count value. 
# It was a prevention of numerical error when we calculate log2FC
sgRNA_readcount[:counts_bin1] -= 0.5;
sgRNA_readcount[:counts_bin2] -= 0.5;
head(sgRNA_readcount) # gotcha?

,gene,barcodeid,counts_bin1,counts_bin2
1,1,1,266.0,117.0
2,1,2,662.0,255.0
3,1,3,244.0,125.0
4,1,4,335.0,127.0
5,1,5,1283.0,405.0
6,1,6,1704.0,614.0
